<a href="https://colab.research.google.com/github/Blackgale/Desafio-Spoof-or-Live/blob/main/C%C3%B3pia_de_Desafio_Most.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import csv
import sys
import numpy as np
import time
import argparse
from sklearn.ensemble import ExtraTreesClassifier

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from PIL import Image
import os

In [ ]:
from google.colab import drive
from PIL import Image
import os

# Montar o Google Drive
drive.mount('/content/drive')

# Definir o caminho para a pasta de imagens no Google Drive
folder_path = '/content/drive/MyDrive/MLChallenge_Dataset/Data'

# Listar os arquivos na pasta
files = os.listdir(folder_path)

# Loop pelos arquivos e ler as imagens
for file in files:
    # Criar o caminho completo para o arquivo
    file_path = os.path.join(folder_path, file)

    # Verificar se o arquivo é uma imagem
    if file_path.endswith('.jpg') or file_path.endswith('.png'):
        # Abrir a imagem usando a biblioteca PIL
        image = Image.open(file_path)

        # Mostrar a imagem
        image.show()


Mounted at /content/drive


In [ ]:
live_folder_path = '/content/drive/MyDrive/MLChallenge_Dataset/Data/live'
spoof_folder_path = '/content/drive/MyDrive/MLChallenge_Dataset/Data/spoof'

In [ ]:
data_transforms = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor()
])

In [ ]:
class LiveSpoofDataset(Dataset):
    def __init__(self, data_folder_path, transform=None):
        self.data_folder_path = data_folder_path
        self.transform = transform

        # Listar os arquivos nas pastas
        self.live_files = []
        self.spoof_files = []
        for subfolder in os.listdir(data_folder_path):
            subfolder_path = os.path.join(data_folder_path, subfolder)
            if os.path.isdir(subfolder_path):
                live_subfolder_path = os.path.join(subfolder_path, 'live')
                if os.path.isdir(live_subfolder_path):
                    for file in os.listdir(live_subfolder_path):
                        file_path = os.path.join(live_subfolder_path, file)
                        self.live_files.append(file_path)

                spoof_subfolder_path = os.path.join(subfolder_path, 'spoof')
                if os.path.isdir(spoof_subfolder_path):
                    for file in os.listdir(spoof_subfolder_path):
                        file_path = os.path.join(spoof_subfolder_path, file)
                        self.spoof_files.append(file_path)

    def __len__(self):
        return len(self.live_files) + len(self.spoof_files)

    def __getitem__(self, idx):
        if idx < len(self.live_files):
            # Ler uma imagem live
            file_path = self.live_files[idx]
            image = Image.open(file_path)
            label = 0 # live
        else:
            # Ler uma imagem spoof
            idx -= len(self.live_files)
            file_path = self.spoof_files[idx]
            image = Image.open(file_path)
            label = 1 # spoof

        # Aplicar transformações na imagem
        if self.transform:
            image = self.transform(image)

        return image, label


In [ ]:
data_folder_path = '/content/drive/MyDrive/MLChallenge_Dataset/Data'
dataset = LiveSpoofDataset(data_folder_path, transform=data_transforms)


In [ ]:
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

In [ ]:
# Criar DataLoaders para os conjuntos de treinamento e teste
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16)

In [ ]:
# Definir o modelo
model = nn.Sequential(
    nn.Conv2d(3, 32, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(2),
    nn.Conv2d(32, 64, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(2),
    nn.Flatten(),
    nn.Linear(64 * 16 * 16, 256),
    nn.ReLU(),
    nn.Linear(256, 1),
    nn.Sigmoid()
)


In [ ]:
# Definir a função de perda e o otimizador
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters())

In [ ]:
# Definir o dispositivo para executar o treinamento
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# Mover o modelo para o dispositivo
model.to(device)

Sequential(
  (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU()
  (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (4): ReLU()
  (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (6): Flatten(start_dim=1, end_dim=-1)
  (7): Linear(in_features=16384, out_features=256, bias=True)
  (8): ReLU()
  (9): Linear(in_features=256, out_features=1, bias=True)
  (10): Sigmoid()
)

In [ ]:
# Treinar o modelo
num_epochs = 10
for epoch in range(num_epochs):
    # Colocar o modelo em modo de treinamento
    model.train()

    # Loop pelos batches de dados de treinamento
    for X_batch, y_batch in train_loader:
        # Mover os dados para o dispositivo
        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device)

        # Calcular a saída do modelo
        outputs = model(X_batch).squeeze()

        # Calcular a perda
        loss = criterion(outputs, y_batch.float())

        # Retropropagar a perda e atualizar os pesos do modelo
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Colocar o modelo em modo de avaliação
    model.eval()

    # Calcular a acurácia no conjunto de teste
    correct = 0
    total = 0
    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            # Mover os dados para o dispositivo
            X_batch = X_batch.to(device)
            y_batch = y_batch.to(device)

            # Calcular a saída do modelo
            outputs = model(X_batch).squeeze()

            # Converter as saídas em previsões de classe (0 ou 1)
            predicted = (outputs > 0.5).long()

            # Calcular o número de previsões corretas
            correct += (predicted == y_batch).sum().item()
            total += y_batch.size(0)

    accuracy = correct / total

    print(f'Epoch: {epoch + 1}/{num_epochs}, Loss: {loss.item():.4f}, Accuracy: {accuracy:.4f}')

Epoch: 1/10, Loss: 0.5380, Accuracy: 0.9025
Epoch: 2/10, Loss: 0.1611, Accuracy: 0.9323
Epoch: 3/10, Loss: 0.0064, Accuracy: 0.9314
Epoch: 4/10, Loss: 0.0095, Accuracy: 0.9371
Epoch: 5/10, Loss: 0.1712, Accuracy: 0.9374
Epoch: 6/10, Loss: 0.0004, Accuracy: 0.9139
Epoch: 7/10, Loss: 0.0002, Accuracy: 0.9113
Epoch: 8/10, Loss: 0.0293, Accuracy: 0.9352
Epoch: 9/10, Loss: 0.0000, Accuracy: 0.9367
Epoch: 10/10, Loss: 0.0000, Accuracy: 0.9342


In [ ]:
# Treinar o modelo
num_epochs = 10
for epoch in range(num_epochs):
    # Colocar o modelo em modo de treinamento
    model.train()

    # Loop pelos batches de dados de treinamento
    for X_batch, y_batch in train_loader:
        # Mover os dados para o dispositivo
        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device)

        # Calcular a saída do modelo
        outputs = model(X_batch).squeeze()

        # Calcular a perda
        loss = criterion(outputs, y_batch.float())

        # Retropropagar a perda e atualizar os pesos do modelo
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Colocar o modelo em modo de avaliação
    model.eval()

    # Calcular a acurácia no conjunto de teste
    correct = 0
    total = 0
    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            # Mover os dados para o dispositivo
            X_batch = X_batch.to(device)
            y_batch = y_batch.to(device)

            # Calcular a saída do modelo
            outputs = model(X_batch).squeeze()

            # Converter as saídas em previsões de classe (0 ou 1)
            predicted = (outputs > 0.5).long()

            # Calcular o número de previsões corretas
            correct += (predicted == y_batch).sum().item()
            total += y_batch.size(0)

    accuracy = correct / total

    print(f'Epoch: {epoch + 1}/{num_epochs}, Loss: {loss.item():.4f}, Accuracy: {accuracy:.4f}')

Epoch: 1/10, Loss: 0.0000, Accuracy: 0.9362
Epoch: 2/10, Loss: 0.0056, Accuracy: 0.9343
Epoch: 3/10, Loss: 0.8275, Accuracy: 0.9278
Epoch: 4/10, Loss: 0.0009, Accuracy: 0.9171
Epoch: 5/10, Loss: 0.0005, Accuracy: 0.9340
Epoch: 6/10, Loss: 0.0000, Accuracy: 0.9240
Epoch: 7/10, Loss: 0.0000, Accuracy: 0.9324
Epoch: 8/10, Loss: 0.0000, Accuracy: 0.9326
Epoch: 9/10, Loss: 0.0001, Accuracy: 0.9388
Epoch: 10/10, Loss: 0.0000, Accuracy: 0.9386


In [ ]:
# Criar conjuntos de teste separados para imagens live e spoof
test_live_dataset = [x for x in test_dataset if x[1] == 0]
test_spoof_dataset = [x for x in test_dataset if x[1] == 1]

# Colocar o modelo em modo de avaliação
model.eval()

# Calcular a acurácia no conjunto de teste live
correct = 0
total = 0
with torch.no_grad():
    for X, y in test_live_dataset:
        # Mover os dados para o dispositivo
        X = X.to(device)
        if not isinstance(y, torch.Tensor):
            y = torch.tensor(y)
        y = y.to(device)

        # Calcular a saída do modelo
        output = model(X.unsqueeze(0)).squeeze()

        # Converter a saída em uma previsão de classe (0 ou 1)
        predicted = (output > 0.5).long()

        # Calcular o número de previsões corretas
        correct += (predicted == y).sum().item()
        total += 1

live_accuracy = correct / total

# Calcular a acurácia no conjunto de teste spoof
correct = 0
total = 0
with torch.no_grad():
    for X, y in test_spoof_dataset:
        # Mover os dados para o dispositivo
        X = X.to(device)
        if not isinstance(y, torch.Tensor):
            y = torch.tensor(y)
        y = y.to(device)

        # Calcular a saída do modelo
        output = model(X.unsqueeze(0)).squeeze()

        # Converter a saída em uma previsão de classe (0 ou 1)
        predicted = (output > 0.5).long()

        # Calcular o número de previsões corretas
        correct += (predicted == y).sum().item()
        total += 1

spoof_accuracy = correct / total

print(f'Live accuracy: {live_accuracy:.4f}, Spoof accuracy: {spoof_accuracy:.4f}')


Live accuracy: 0.8722, Spoof accuracy: 0.9717


In [ ]:
def predict_image(image_path, model, transform, device):
    # Ler a imagem do arquivo
    image = Image.open(image_path)

    # Aplicar transformações na imagem
    if transform:
        image = transform(image)

    # Mover a imagem para o dispositivo
    image = image.to(device)

    # Colocar o modelo em modo de avaliação
    model.eval()

    # Calcular a saída do modelo
    output = model(image.unsqueeze(0)).squeeze()

    # Converter a saída em uma previsão de classe (0 ou 1)
    y_pred = (output > 0.5).long().item()

    return y_pred

image_path = '/content/Portrait_female_Midjourney.jpg'
y_pred = predict_image(image_path, model, data_transforms, device)

if y_pred == 0:
    print('The image is predicted to be live')
else:
    print('The image is predicted to be spoof')

The image is predicted to be spoof
